In [8]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob as glob
import datetime as dt

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io import shapereader
import cartopy.io.img_tiles as cimgt

from sklearn.linear_model import QuantileRegressor
import sklearn

from metpy.calc import dewpoint_from_relative_humidity
from metpy.units import units
import metpy

# Making daily Max Heat Metrics

### Loading in datasets

In [2]:
stations = ["CMI","LLC","SIU"]
station_ds = []

def parse_and_shift(s):
    # parse into a Timestamp, then subtract 5 hours
    return pd.to_datetime(s) + pd.Timedelta(hours=5)

for station in stations:
    df = pd.read_csv(
        f"hourly/{station}_4-17-2025.csv",
        parse_dates=["Date & Time (CST)"],
        date_parser=parse_and_shift
    )

    df = df.set_index(["Station", "Date & Time (CST)"])
    
    ds = df.to_xarray()
    
    ds = ds.rename({
        "Station": "station",
        "Date & Time (CST)": "time",
        "Wind Speed (mph)": "wind_speed",
        "Air Temperature (deg C)": "t2m",
        "Relative Humidity (%)": "rh",
    })
    
    station_ds.append(ds)

ds = xr.concat(station_ds, dim='station')

# clean data

# change 9999 to nan
ds = ds.where(ds != 9999, other=np.nan) 

# only select summer temperature
summer_mask = ds.time.dt.month.isin([6, 7, 8])
ds = ds.sel(time=summer_mask)

/tmp/ipykernel_883/746704750.py:9: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(
/tmp/ipykernel_883/746704750.py:9: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(
/tmp/ipykernel_883/746704750.py:9: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(


In [5]:
ds

<xarray.Dataset> Size: 6MB
Dimensions:     (station: 3, time: 79488)
Coordinates:
  * time        (time) datetime64[ns] 636kB 1989-06-01 ... 2024-08-31T23:00:00
  * station     (station) object 24B 'Champaign' 'Springfield' 'Carbondale'
Data variables:
    wind_speed  (station, time) float64 2MB 5.9 4.6 4.3 4.6 ... nan nan nan nan
    t2m         (station, time) float64 2MB 28.4 27.1 26.3 ... 23.2 23.0 23.2
    rh          (station, time) float64 2MB 75.6 82.7 87.4 ... 87.4 92.0 91.1

### Calculate the Dewpoint temperature of ICN data

In [9]:
rh_ds = ds.assign(d2m=dewpoint_from_relative_humidity(ds.t2m * units.degC, ds.rh * units.percent).metpy.dequantify())
rh_ds

<xarray.Dataset> Size: 8MB
Dimensions:     (station: 3, time: 79488)
Coordinates:
  * time        (time) datetime64[ns] 636kB 1989-06-01 ... 2024-08-31T23:00:00
  * station     (station) object 24B 'Champaign' 'Springfield' 'Carbondale'
Data variables:
    wind_speed  (station, time) float64 2MB 5.9 4.6 4.3 4.6 ... nan nan nan nan
    t2m         (station, time) float64 2MB 28.4 27.1 26.3 ... 23.2 23.0 23.2
    rh          (station, time) float64 2MB 75.6 82.7 87.4 ... 87.4 92.0 91.1
    d2m         (station, time) float64 2MB 23.68 23.91 24.04 ... 21.63 21.67

In [10]:
rh_ds.to_netcdf('hourly/ICN_dewpoint_1989-2024_2025-04-17.nc')

### calculating different metrics

In [9]:
# This function calculate webulb temperature. It asks for temperature in celcius, and relative humidity in %
# ex: tmp = 20, rh = 50
def Wetbulb(tmp, rh):
    wetbulb_tmp = ( tmp * np.arctan(0.151977 * (rh + 8.313659)**(1/2)) ) + np.arctan(tmp + rh) - np.arctan(rh - 1.676331) + (0.00391838 * (rh)**(3/2)) * np.arctan(0.023101 * rh) - 4.686035
    return wetbulb_tmp

def Heat_Index(T, RH):
    """
    https://www.wpc.ncep.noaa.gov/html/heatindex_equation.shtml

    Calculates heat index for an array
    
    Inputs:
        RH (DataArray) - Should be in decimal format
        T  (DataArray) - Should be in Kelvins
        
    Outputs:
        hi_alone (DataArray) - Heat index array (in F)
    """
    # Convert to Fahrenheit
    T_F = ((T - 273.15) * 1.8) + 32

    # Convert to relative humidity
    RH_p = RH * 100
    RH_p = RH_p.rename('relative_humidity')
    
    # Standard heat index
    heat_index = 0.5 * (T_F + 61.0 + ((T_F-68.0)*1.2) + (RH_p*0.094))
    heat_index = heat_index.rename('heat_index')

    # Combining temperature, relative humidity, and heat index into a dataset
    hi_set = xr.combine_by_coords((heat_index,T_F,RH_p))
        
    # Heat index for heat index above 80
    heat_index_80 = (-42.379 + 2.04901523*T_F + 10.14333127*RH_p - 0.22475541*T_F*RH_p 
          - 6.83783e-3*T_F**2 - 5.481717e-2*RH_p**2 + 1.22874e-3*T_F**2*RH_p 
          + 8.5282e-4*T_F*RH_p**2 - 1.99e-6*T_F**2*RH_p**2)
    hi_set['heat_index>80'] = heat_index_80
    
    # Replacing heat indices above 80 with the new equation
    hi_set['heat_index'] = xr.where(hi_set['heat_index']>80,
                                    hi_set['heat_index>80'],
                                    hi_set['heat_index']
                                    )
    
    # Heat index for relative humidity under 13% and temps between 80 and 112 F
    heat_index_13 = heat_index_80 - ((13-RH_p)/4) * np.sqrt((17 - abs(T_F - 95))/17)
    hi_set['heat_index_RH<13'] = heat_index_13
    
    hi_set['heat_index'] = xr.where(((hi_set['relative_humidity']<13) & 
                                         (hi_set['t2m']>80) & 
                                         (hi_set['t2m']<112)),
                                    hi_set['heat_index_RH<13'],
                                    hi_set['heat_index'])
    
    # Heat index for relative humidity over 85% and temps between 80 and 87 F
    heat_index_85 = heat_index_80 + ((RH_p-85)/10) * ((87-T_F)/5)
    hi_set['heat_index_RH>85'] = heat_index_85
    hi_set['heat_index'] = xr.where(((hi_set['relative_humidity']>85) & 
                                         (hi_set['t2m']>80) & 
                                         (hi_set['t2m']<87)),
                                    hi_set['heat_index_RH>85'],
                                    hi_set['heat_index'])
    
    # Picking out the heat index dataarray alone
    hi_alone = hi_set['heat_index']

    return hi_alone

In [10]:
heat_ds = ds

# Assigning variables
heat_ds = heat_ds.assign(tmp_wb=Wetbulb(heat_ds.t2m,heat_ds.rh)) 

heat_ds = heat_ds.assign(heat_index=Heat_Index((heat_ds.t2m + 273.15),heat_ds.rh / 100)) 

# Make Daily highs
max_tmp = heat_ds.t2m.groupby(heat_ds.time.dt.date).max()
max_wb  = heat_ds.tmp_wb.groupby(heat_ds.time.dt.date).max()
max_hi  = heat_ds.heat_index.groupby(heat_ds.time.dt.date).max()

/data/keeling/a/kchoo3/anaconda3/envs/myenv/lib/python3.12/site-packages/xarray/core/computation.py:825: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


In [11]:
# adding attributes and descriptions
max_tmp.attrs['units'] = 'Celsius'
max_tmp.attrs['description'] = 'Temperature in Celsius'

max_wb.attrs['units'] = 'Celsius'
max_wb.attrs['description'] = 'Wet-bulb temperature in Celsius'

max_hi.attrs['units'] = 'Fahrenheit'
max_hi.attrs['description'] = 'NOAAs Heat index'


ds = xr.Dataset({
    't2m': max_tmp,
    'wb': max_wb,
    'hi': max_hi
}, attrs={
    'title': 'Combined Maximum Temperature Data',
    'summary': 'ICN Dataset containing maximum daily of temperature, wet-bulb temperature, and heat index from 1989-2024 in June-July-August.'}
)
# ds.to_netcdf('daily_max.nc')
ds['date'] = np.array(ds['date'].values, dtype='datetime64[ns]')

ds.to_netcdf('ICN_heat_metrics_1989-2024_2025-04-17_daily_max.nc')

In [60]:
ds

<xarray.Dataset> Size: 265kB
Dimensions:  (station: 3, date: 3312)
Coordinates:
  * station  (station) object 24B 'Champaign' 'Springfield' 'Carbondale'
  * date     (date) datetime64[ns] 26kB 1989-06-01 1989-06-02 ... 2024-08-31
Data variables:
    t2m      (station, date) float64 79kB 28.7 27.5 26.3 22.3 ... 35.8 33.4 29.4
    wb       (station, date) float64 79kB 26.07 21.48 23.78 ... 24.7 25.03 23.86
    hi       (station, date) float64 79kB -455.3 -459.6 -460.5 ... -447.7 -454.8
Attributes:
    title:    Combined Maximum Temperature Data
    summary:  ICN Dataset containing maximum daily of temperature, wet-bulb t...

# Quantile Regression

In [12]:
daily_ds = xr.open_dataset('ICN_heat_metrics_1989-2024_2025-04-17_daily_max.nc')
daily_ds

<xarray.Dataset> Size: 265kB
Dimensions:  (station: 3, date: 3312)
Coordinates:
  * station  (station) <U11 132B 'Champaign' 'Springfield' 'Carbondale'
  * date     (date) datetime64[ns] 26kB 1989-06-01 1989-06-02 ... 2024-08-31
Data variables:
    t2m      (station, date) float64 79kB ...
    wb       (station, date) float64 79kB ...
    hi       (station, date) float64 79kB ...
Attributes:
    title:    Combined Maximum Temperature Data
    summary:  ICN Dataset containing maximum daily of temperature, wet-bulb t...

In [16]:
def quantile_regression(y, X):
    """
    Use sklearn's QuantileRegressor to
    Returns an array with [slope, intercept].
    
    Parameters
    ----------
    y : array, shape (time,) values at one grid point.
    X : array, should be the date values
    """
    qr = QuantileRegressor(quantile=0.50, alpha=0)
    qr.fit(X, y)
    return np.array([qr.coef_[0], qr.intercept_])

# loading data
ds = daily_ds

X = daily_ds.date.values[:, np.newaxis].astype('datetime64[D]').astype('int64')

variable = 'hi'

ds = ds[variable]
ds = ds.fillna(ds.mean())


result = xr.apply_ufunc(
    quantile_regression,
    ds,  # the data variable to process
    input_core_dims=[["date"]],  
    kwargs={'X': X},  
    vectorize=True,
    dask='parallelized', 
    output_core_dims=[["params"]],
    output_dtypes=[float]
)

In [17]:
slope = result.sel(params=0)
intercept = result.sel(params=1)

quantile = 50

slope.attrs['description'] = f'The beta values. The trend of the {quantile}th linear regression'
intercept.attrs['description'] = f'The alpha values. The baseline {quantile}qr temperature'

save_ds = xr.Dataset({
    'slope': slope,
    'intercept': intercept
}, attrs={
    'title': f'{quantile}th QuantileRegressor of {variable}',
    'summary': f'Dataset the slope and intersect of {quantile}th QuantileRegressor of {variable}.'}
)
save_ds.to_netcdf(f'qr/illinois/{quantile}/hi_overall_{quantile}qr.nc')

## Validation

In [5]:
icn_dir = '/data/cristi/a/kchoo3/projects/cdds-heat-metrics/ICN/qr/illinois'
quantile = '95'

icn_t2m_qr = xr.open_dataset(f'{icn_dir}/{quantile}/t2m_overall_{quantile}qr.nc')
icn_wb_qr = xr.open_dataset(f'{icn_dir}/{quantile}/wb_overall_{quantile}qr.nc')
icn_hi_qr = xr.open_dataset(f'{icn_dir}/{quantile}/hi_overall_{quantile}qr.nc')

In [10]:
icn_hi_qr.slope.values * 35

array([-0.00086922, -0.00067439,  0.00054049])